# Final Report: 06/19/2019

## Brief: What was done previously
Previous work discovered multiple mechanisms to improve the accuracy of prediction. Here we quantify the performance of those methods and test them against a standard baseline. Additionally, we quantify learned model's resistence to noise and missing observations.

## Hypothesis

1. Hypothesis 1: Deep recurrent networks are tolerant of sensor noise below a certain magnitude (fixed gaussians per pixel noise)
2. Hypothesis 2: Deep recurrent networks are tolerant of missing samples (random per pixel dropout)
3. Hypothesis 3: Deep recurrent networks are tollerant of missing observations (random per history dropout)




## Summary of Main Results and Discussions



### Experiment 1 results and discussion
Leaned models handle noise suppression impresivly well, dealing with noise with standard deviation twice as large as the range of the sample data. 

### Experiment 2: results and discussion
Put main result and conclusions here. Discuss importance/impact in terms of the project goals.


## Plan for next effort
What will be tested/extended from this week?
    

    


In [5]:
# import packages 
import importlib
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import os

# Hypothesis 1

Deep recurrent networks are tolerant to sensor noise below a certain magnitude

## Validation accuracy over increasing fixed gaussian sensor noise
We add decreasing magnitudes of gaussian noise to the input and predict the clean future signal. We expect, for
high levels of noise, the model to over-fit to the noise. However, after some threshold we expect the model to learn to recover from small perturbations by learning the underlying distribution.

In [1]:
# Gaussian noise study (fixed input noise)
import numpy as np
import tensorflow as tf

from src.predict_turbulence_recurrent import train
from src.dataLoader.turbulence import Turbulence, RANDOM_SEED, LARGE_DATASET

# Use a fixed seed for noise    
np.random.seed(RANDOM_SEED)

noise = [0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0]
noise.reverse()

for scale in noise:
    print(scale)
    noise_data = np.random.normal(size=(360, 279, 1000), scale=scale)
    
    loader = Turbulence(pred_length=20, dataset_idx=LARGE_DATASET, input_noise=noise_data, debug=False)
    
    train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=50000, net_name='lstm_3_cells_p20_static_noise_{}'.format(scale))
    
    tf.reset_default_graph()


C:\Users\brandon\source\orbitalMechanics
0
Instructions for updating:
Colocations handled automatically by placer.
Input shape: (20, ?, 2500)
Output shape: (20, ?, 50, 50)
Encoder input shape: [20, None, 2500]
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Padded input shape: [20, 64, 2500]
[20, 64, 250]
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Summary name Loss Histogram is illegal; using Loss_Histogram instead.
INFO:tensorflow:Summary name Mean Abs Error is illegal; using Mean_Abs_Error instead.
( 0.2882854 0 )
( 0.009317085 500 )
0.0074923905 1000

0.0009851957 34000
0.0010684747 34500
0.001039591 35000
0.00097291014 35500
0.000976496 36000
0.001038831 36500
0.0009173796 37000
0.00080046186 37500
0.0008736703 38000
0.0007578315 38500
0.0008269094 39000
0.0008396534 39500
0.00096515985 40000
0.0009528528 40500
0.0007295755 41000
0.00094498845 41500
0.00082459813 42000
0.0007952562 42500
0.00084162207 43000
0.00064889033 43500
0.00085401646 44000
0.0007259489 44500
0.00076178304 45000
0.00073619746 45500
0.00062669406 46000
0.00081771147 46500
0.00085382926 47000
0.00084951986 47500
0.0006922747 48000
0.0007537976 48500
0.0007451745 49000
0.00079924316 49500
0.00068149564 50000
0.01
Input shape: (20, ?, 2500)
Output shape: (20, ?, 50, 50)
Encoder input shape: [20, None, 2500]
Padded input shape: [20, 64, 2500]
[20, 64, 250]
INFO:tensorflow:Summary name Loss Histogram is illegal; using Loss_Histogram instead.
INFO:tensorflow:Summary name Mean Abs Error is illegal; using Mean_Abs_Error instead.
( 0.28724465 0 )
( 0.009149698 500 )
0.

0.001956491 16500
0.0017040834 17000
0.001764122 17500
0.001814051 18000
0.0018678379 18500
0.0016765594 19000
0.0017129041 19500
0.0015826632 20000
0.0014578974 20500
0.0016860779 21000
0.0014549509 21500
0.0012978254 22000
0.0014636128 22500
0.0014392918 23000
0.0012625053 23500
0.0012415893 24000
0.0011831783 24500
0.0011620543 25000
0.0012105368 25500
0.0011577783 26000
0.0011740477 26500
0.0011759542 27000
0.0011189929 27500
0.0012449645 28000
0.0011676874 28500
0.0011701994 29000
0.0009985236 29500
0.0011644003 30000
0.0010583892 30500
0.0009604003 31000
0.0010546232 31500
0.001088419 32000
0.0011103603 32500
0.00090387714 33000
0.0011823507 33500
0.0010554482 34000
0.0010094868 34500
0.0009925667 35000
0.0009281368 35500
0.0009325047 36000
0.0009847735 36500
0.00089459243 37000
0.0008878709 37500
0.000994804 38000
0.0008113062 38500
0.0008726503 39000
0.00081579667 39500
0.00086884433 40000
0.00095980987 40500
0.00076529285 41000
0.00093624945 41500
0.0007179229 42000
0.00079554

## Explore results

In this test given a 3 layer encoder/decoder with 250 units per layer, we see that the performance of the model is 
resistant to up to 5% noise without any degradation in predictive power. Even with 100% noise the model learned to 
reject some amount of noise and 

In [6]:
# Compare accuracy of model with increasing fixed noise
import os
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.offline.init_notebook_mode(connected=True)
    
# Compare MSE vs magnitude of noise
noise = [2, 1, 0.75, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.0001, 0]
noise = [noise[-(i+1)] for i in range(len(noise))]
# validation_accuracy = [2.0371e-3, 8.1668e-4, 5.475e-4, 2.8722e-4, 1.0855e-4, 4.5386e-5, 3.1529e-5, 2.7214e-5, 2.0735e-5]
train_accuracy = [1.836e-5, 1.8441e-5, 2.0871e-5, 2.0525e-5, 2.0918e-5, 2.2431e-5, 2.2850e-5, 3.6688e-5, 6.5500e-5, 7.1376e-5, 7.3004e-5, 7.1560e-5, 1.1372e-4]
validation_accuracy = [1.932e-5, 1.9536e-5, 2.1711e-5, 2.1794e-5, 2.1213e-5, 2.3923e-5, 2.4532e-5, 4.2984e-5, 1.0194e-4, 2.9481e-4, 5.4323e-4, 7.5883e-4, 2.0583e-3]

# Create a trace
trace1 = go.Scatter(
    x = noise,
    y = validation_accuracy,
    name="validation"
)
trace2 = go.Scatter(
    x = noise,
    y = train_accuracy,
    name="train"
)

data = [trace1]
layout = go.Layout(
    title="Magnitude of Sensor Noise vs Prediction Accuracy",
    xaxis=dict(
        type='log',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Standard Deviation of Added Gaussian Noise',
    ),
    yaxis=dict(
        type='log',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean Squared Validation Error over 20 Steps',
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='static_noise_model')



c:\python\python36\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [2]:
data = [trace1, trace2]
layout = go.Layout(
    title="Magnitude of Sensor Noise vs Prediction Accuracy",
    xaxis=dict(
        type='log',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Standard Deviation of Added Gaussian Noise',
    ),
    yaxis=dict(
        type='log',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean Squared Validation Error over 20 Steps',
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='static_noise_model')

NameError: name 'trace1' is not defined

In [8]:
# Visualize data
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Make sub-plots for input sequence
i = 0
plt.rc('text', usetex=True)
fig, plots = plt.subplots(1, 8, figsize=(20, 4))

exp_root = './experiments/turbulence/recurrent_scaled_mse'
exp_folders = ['lstm_3_cells_20_static_noise_{}_velocity_and_vorticity_field_1200s.mat_lr0.0005'.format(n) for n in noise if n != 0.0001]
titles = ['Sequence + N $\sig = {} \mu = 0$'.format(n) for n in noise if n != 0.0001]
for directory in exp_folders:

    for file in os.listdir(os.path.join(exp_root, directory)):
        if i < 8:
            if file.endswith('.npz') and file.startswith('inputs'):
                file_path = os.path.join(exp_root, directory, file)
                with np.load(file_path) as foo:
                    plots[i].imshow(np.reshape(foo['100000'][0,0,:], (50,50)))
                    plots[i].set_title(titles[i])
                    i += 1
                    print(directory)

for ax in fig.get_axes():
    ax.label_outer()
    
plt.show()
                
                



lstm_3_cells_20_static_noise_0_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.0025_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.005_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.01_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.025_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.05_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.1_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.25_velocity_and_vorticity_field_1200s.mat_lr0.0005


RuntimeError: latex was not able to process the following string:
b'Sequence + N $\\\\sig = 0 \\\\mu = 0$'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.7000 64-bit)
entering extended mode
(C:/Users/brandon/.matplotlib/tex.cache/4e09206970ed48dd24e0e16b31502d57.tex
LaTeX2e <2018-12-01>
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\article.cls
"
Document Class: article 2018/09/03 v1.4i Standard LaTeX document class
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\size10.clo"
))
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/type1cm\type1cm.
sty")
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\textcomp.st
y"
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.def"
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.dfu"
)))
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\in

<Figure size 1440x288 with 8 Axes>

In [48]:
#     try:
#         with np.load(os.path.join(exp_dir, directory, 'label_100000_0.0008166771149262786.npy')) as labels:
#             with np.load(os.path.join(exp_dir, directory, 'pred_100000_0.0008166771149262786.npy')) as predictions:
#                 foo = {key:labels[key].item() for key in labels}
#                 for arr in predictions.keys():
#                     print(predictions[arr].item())
#                     for key, value in dict(predictions[arr].tolist()).items():
#                         print (key,':', value.shape)
#                     # print(dict(labels))
#                     # print(dict(labels[key].tolist())['2000'])
#                     # print(predictions[key])
#     except FileNotFoundError:
#         continue
#     
# 
# trace = go.Heatmap(
#     z=frame,
#     colorscale='Viridis')
# data=[trace]
# py.iplot(data, filename='basic-heatmap')

# Hypothesis 2

Sample level dropout

# Test 1

Description of test 1 of hypothesis 2.


In [ ]:
# Missing samples study (random missing pixels)
import numpy as np
import tensorflow as tf

from src.predict_turbulence_recurrent import train
from src.dataLoader.turbulence import Turbulence, RANDOM_SEED, LARGE_DATASET

# Use a fixed seed  
np.random.seed(RANDOM_SEED)

for s in [0.9, 0.75, 0.5, 0.25, 0.1]:
        
    loader = Turbulence(pred_length=20, dataset_idx=LARGE_DATASET, debug=False)
    
    train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=100000, pixel_dropout=s,
          net_name='lstm_3_cells_20_pixel_dropout_{}'.format(s))
    
    tf.reset_default_graph()

In [1]:
# Test single layer
# Gaussian noise study (fixed input noise)
import numpy as np
import tensorflow as tf

from src.predict_turbulence_recurrent import train
from src.dataLoader.turbulence import Turbulence, LARGE_DATASET


loader = Turbulence(pred_length=20, dataset_idx=LARGE_DATASET, input_noise=None, debug=False)

train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=50000, net_name='singe_layer')

tf.reset_default_graph()


C:\Users\brandon\source\orbitalMechanics
Instructions for updating:
Colocations handled automatically by placer.
Input shape: (20, ?, 2500)
Output shape: (20, ?, 50, 50)
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Summary name Loss Histogram is illegal; using Loss_Histogram instead.
INFO:tensorflow:Summary name Mean Abs Error is illegal; using Mean_Abs_Error instead.


InvalidArgumentError: Cannot parse tensor from proto: dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 50000
  }
  dim {
    size: 50000
  }
}
float_val: 0

	 [[node dense/kernel/Adam_1/Initializer/zeros (defined at C:\Users\brandon\source\orbitalMechanics\src\predict_turbulence_recurrent.py:327) ]]

Caused by op 'dense/kernel/Adam_1/Initializer/zeros', defined at:
  File "c:\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\python\python36\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "c:\python\python36\lib\site-packages\tornado\platform\asyncio.py", line 1